In [1]:
import pandas as pd
import time
import boto3

In [2]:
transcribe = boto3.client('transcribe')

In [8]:
def check_job_name(job_name):
    job_verification = True
    # all the transcriptions
    existed_jobs = transcribe.list_transcription_jobs()
    for job in existed_jobs['TranscriptionJobSummaries']:
        if job_name == job['TranscriptionJobName']:
            job_verification = False
            break
    if job_verification == False:
        command = input(job_name + " has existed. \nDo you want to override the existed job (Y/N): ")    
        if command.lower() == "y" or command.lower() == "yes":                
            transcribe.delete_transcription_job(TranscriptionJobName=job_name)
        elif command.lower() == "n" or command.lower() == "no":      
            job_name = input("Insert new job name? ")      
            check_job_name(job_name)
        else:
            print("Input can only be (Y/N)")
            command = input(job_name + " has existed. \nDo you want to override the existed job (Y/N): ")
    return job_name

In [9]:
def amazon_transcribe(audio_file_name):
    job_uri = "s3://upacare-dev-ml/1249120_44142156_72079889.wav"
    # Usually, I put like this to automate the process with the file name
    # "s3://bucket_name" + audio_file_name  
    # Usually, file names have spaces and have the file extension like .mp3
    # we take only a file name and delete all the space to name the job
    job_name = (audio_file_name.split('.')[0]).replace(" ", "")  
    # file format  
    file_format = audio_file_name.split('.')[1]
    
    # check if name is taken or not
    job_name = check_job_name(job_name)
    transcribe.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={'MediaFileUri': job_uri},
        MediaFormat = file_format,
        LanguageCode='en-US')
    
    while True:
        result = transcribe.get_transcription_job(TranscriptionJobName=job_name)
        if result['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
        time.sleep(15)
    if result['TranscriptionJob']['TranscriptionJobStatus'] == "COMPLETED":
        data = pd.read_json(result['TranscriptionJob']['Transcript']['TranscriptFileUri'])
    return data['results'][1][0]['transcript']

In [10]:
res = amazon_transcribe("1249120_44142156_72079889.wav")

/tmp/ipykernel_109803/4254965779.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return data['results'][1][0]['transcript']


'My face is all broken out with pimples. Bye bye.'

In [13]:
### Medical

### Medical

In [3]:
def check_medical_job_name(job_name):
    job_verification = True
    # all the transcriptions
    existed_jobs = transcribe.list_medical_transcription_jobs()
    for job in existed_jobs['MedicalTranscriptionJobSummaries']:
        if job_name == job['MedicalTranscriptionJobName']:
            job_verification = False
            break
    if job_verification == False:
        command = input(job_name + " has existed. \nDo you want to override the existed job (Y/N): ")    
        if command.lower() == "y" or command.lower() == "yes":                
            transcribe.delete_medical_transcription_job(MedicalTranscriptionJobName=job_name)
        elif command.lower() == "n" or command.lower() == "no":      
            job_name = input("Insert new job name? ")      
            check_job_name(job_name)
        else:
            print("Input can only be (Y/N)")
            command = input(job_name + " has existed. \nDo you want to override the existed job (Y/N): ")
    return job_name

In [4]:
def amazon_transcribe_medical(audio_file_name):
    job_uri = "s3://upacare-dev-ml/"+audio_file_name
    # Usually, I put like this to automate the process with the file name
    # "s3://bucket_name" + audio_file_name  
    # Usually, file names have spaces and have the file extension like .mp3
    # we take only a file name and delete all the space to name the job
    job_name = (audio_file_name.split('.')[0]).replace(" ", "")  
    # file format  
    file_format = audio_file_name.split('.')[1]
    
    # check if name is taken or not
    job_name = check_medical_job_name(job_name)
    transcribe.start_medical_transcription_job(
        MedicalTranscriptionJobName=job_name,
        Media={'MediaFileUri': job_uri},
        MediaFormat = file_format,
        LanguageCode='en-US',
        Specialty = 'PRIMARYCARE',
        Type = 'CONVERSATION',
        OutputBucketName="upacare-dev-ml"
        )
    
    while True:
        result = transcribe.get_medical_transcription_job(MedicalTranscriptionJobName=job_name)
        if result['MedicalTranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
            break
        time.sleep(15)
    print(result['MedicalTranscriptionJob']['Transcript']['TranscriptFileUri'])

In [14]:
res = amazon_transcribe_medical("D0421-S1-T05.wav")
print(res)

https://s3.us-east-1.amazonaws.com/upacare-dev-ml/medical/D0421-S1-T05.json
None


In [6]:
s3_client = boto3.client("s3")


In [15]:
s3_client.download_file(
                    Filename="/home/fm-pc-lt-228/Desktop/upacare/speech-to-speech/speech_to_text/D0421-S1-T05.json",
                    Bucket= "upacare-dev-ml",
                    Key= "medical/D0421-S1-T05.json"
                )